# Build a language model based on wikipedia in you language 

The notebooke includes the whole process but you will need to help the process if the following problem happens:  
-you internet connection is interrupted (stage 1)

-you run out of diskspace

-because of the huge memory consumption. creatig the databunch for the training requires lots of memory


In order ot handle these problemn the notebook is divided into stages. If the process is failes in a stage then you can resume the processing from the beginning of that stage by:
-restarting the kernel "Kernel"/Restart & Clear Output

-running the cells in stage 0: initialization

-running the cells from the start of the stage that failed 

# Stage 0: Initialisation
lang: You must set the parameter "lang" to the language you want to build a model for. Fx:

fr: for french

en: for english

de:for german

da: for danish 

etc.

pathData: You must se the location where you want your data stored using the parameter pathData. Consider using a ssd-rive

In [ ]:
lang="en"

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai.text import * 

from IPython.display import display, HTML
from pathlib import *
import re
import shutil

In [ ]:
pathData       = Path("../../data/nlp")
path           = pathData / lang
pathDump       = path/"wiki-dump"
pathJson       = path/"wiki-json"

pathTrainValid = path/"wiki-train_valid"
pathTxt        = pathTrainValid/"txt"
pathToks       = pathTrainValid/"toks"
pathcsv        = pathTrainValid/"wiki.csv"

cache_name   = "sp-model"
pathVocab    = pathTrainValid / cache_name

# requried libraries: 
conda install -c anaconda psutil 

conda install -c anaconda git 

#conda install -c menpo wget 
conda install curl


# Stage 1: download the selected language from wikipedia and convert the articles to json

In [ ]:
if len(list(pathDump.glob("*.bz2")))==0:
    pathDump.mkdir(parents=True,exist_ok=True)
    #fn  = f"{lang}wiki-latest-pages-articles.xml.bz2"
    fn  = f"{lang}wiki-latest-pages-articles-multistream.xml.bz2"
    url = f"https://dumps.wikimedia.org/{lang}wiki/latest/{fn}"
    #cmd = f"wget -c --no-check-certificate --show-progress {str(url)} -P {str(pathDump)}"
    cmd = f'curl -k -C - -o "{str(pathDump/fn)}" "{str(url)}"'
    print(f"If the command fails in the notebook then copy the command and run it in the terminal:{cmd}")
    ! $cmd

# Stage 2: Convert wikipedia dump to articles in json

In [ ]:
pathWikiExtractor = Path("../wikiextractor")

In [ ]:
if not pathJson.exists():
    !git clone https://github.com/attardi/wikiextractor.git $pathWikiExtractor
    cmd = f"cd {str(pathWikiExtractor)} && python setup.py install"
    ! $cmd

In [ ]:
%%time
if not pathJson.exists():
    #extracting the json-files. This takes about 1 hour for french with files read from and saved to a ssd hardrive 
    fn_wikidump = list(pathDump.iterdir())[0]
    cmd = f"cd {str(pathWikiExtractor)} && python WikiExtractor.py -o {str(pathJson)} --json --processes {defaults.cpus} -q  {str(fn_wikidump)}"
    print(f"If WikiExtractor fails in the notebook then copy the command and run it in the terminal:{cmd}")
    ! $cmd

# Stage 3: building a vocabulary using sentencepiece

Now we separate the title of the wikipedia section from text section that we keep. 

In order to makes a first reduction on the number of section we clean the text with the preprocessing rules from fastai and ignore text with less than "minWords"

You must set the lenght of the shortes sections you want to keep using the parameter "minWords"

In [ ]:
minWords  = 10
chunksize = 25000

In [ ]:
!pip install sentencepiece

In [ ]:
from fastai_sentencepiece import *
from filetokenizer import *
from languagemodelloader import *

In [ ]:
if not pathTxt.exists():
    swm = SentencepieceWikiModel(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab, minWords=minWords )    
    %time swm.wikijson2TrainingData()   


In [ ]:
if not (pathVocab/"m.model").exists():
    swm = SentencepieceWikiModel(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab, minWords=minWords)
    %time swm.trainVocabulary()

In [ ]:
if not (pathVocab/"itos.pkl").exists():
    swm = SentencepieceWikiModel(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab, minWords=minWords)
    swm.convertVocab()

# Show some examples using the vocabulary

In [ ]:
swm = SentencepieceWikiModel(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab, minWords=minWords)
sp = spm.SentencePieceProcessor()
sp.Load(str(pathVocab / "m.model"))
print("1: Size of vocabulary:",sp.GetPieceSize())
sentence = "She is tall. He is small"
print("2:", sp.EncodeAsPieces(sentence))
print("3:", sp.EncodeAsIds(sentence))


In [ ]:
print(f"Control symbol")
for s in ["<unk>"]: print(f"{s}({sp.PieceToId(s)})")

print(f"\nuser_defined_symbols")
for s in swm.getUserdefinedSymbols():print(f"{s}({sp.PieceToId(s)})")

# Prepare training of the language model

# Create Tokenizer
The sentencepiece vocabulary was trained in Train Sentencepiece tokenizer.ipynb.

Here we will make a BasicTokenizer from Sentencepiece

In [ ]:
spt_func  = partial(SentencepieceTokenizer.create, pathVocab=pathVocab)
spt_func.__name__ = SentencepieceTokenizer.__name__
spt       = spt_func(lang="en")
tokenizer = Tokenizer(spt_func,"en")

pad_idx   = spt.vocab().numericalize([text.transform.PAD])[0]
vocab,max_vocab  = spt.vocab(), len(spt.vocab().itos)

print(tokenizer)
print("size og vocabulary:", max_vocab)
print("pad_idx:",pad_idx)

print(spt.vocab().numericalize( ["<unk>" ,"xxbos" ,"xxpad" ,"xxmaj" ,"xxup" ,"xxrep" ,"xxwrep", "xxfld"]  ))
sentence = ["She is tall.", "He is small"]
tokenizer._process_all_1(sentence)

print("vocab:",vocab)

# LM Training
Set the minimum number of tokens for the sections that we shall retain

In [ ]:
#Discard section with kess than "minTok" tokens
minToks = 10

In [ ]:
pad_idx   = spt.vocab().numericalize([text.transform.PAD])[0]
vocab,max_vocab = spt.vocab(), len(spt.vocab().itos)
trainTokenizer = FileTokenizer(pathToks/"train", spt_func,"en",vocab,minToks=minToks,n_cpus=max(defaults.cpus-1,1))
validTokenizer = FileTokenizer(pathToks/"valid", spt_func,"en",vocab,minToks=minToks,n_cpus=max(defaults.cpus-1,1))

print(trainTokenizer)
print("size og vocabulary:", max_vocab)
print("pad_idx:",pad_idx)

print(spt.vocab().numericalize( ["<unk>" ,"xxbos" ,"xxpad" ,"xxmaj" ,"xxup" ,"xxrep" ,"xxwrep", "xxfld"]  ))
#sentence = ["She is tall.", "He is small"]
#tokenizer._process_all_1(sentence)

In [ ]:
files   = np.asarray( list(pathTxt.glob("*.txt")) )
nrows   = len(files)
split   = 0.2
splitindex, index = int(nrows*split+.5), np.random.permutation(np.arange(nrows)) 

chunksize=0

trainList = TextList( files[:-splitindex], vocab=vocab, pad_idx=pad_idx, 
                      processor=[FileTokenizeProcessor(tokenizer=trainTokenizer, 
                                                       chunksize=chunksize, mark_fields=False)])

validList = TextList( files[-splitindex:], vocab=vocab, pad_idx=pad_idx, 
                      processor=[FileTokenizeProcessor(tokenizer=validTokenizer, 
                                                       chunksize=chunksize, mark_fields=False)])


In [ ]:
if not (pathToks/"train").exists():
    %time trainList.process()
    trainList=None

In [ ]:
if not (pathToks/"valid").exists():
    %time p = validList.process()
    validList=None

In [ ]:
%time trainIDS = trainTokenizer.getIds()

In [ ]:
%time validIDS=validTokenizer.getIds()

In [ ]:
%%time
print(f"Traning:    number of arrays:{len(trainIDS)} - number of ids:{np.sum([len(ids) for ids in trainIDS])}")
print(f"Validation: number of arrays:{len(validIDS)} - number of ids:{np.sum([len(ids) for ids in validIDS])}")

#Analyse the distribution of the legnth of tokens sequences in the ragged/jagged array of tokes
sectionlengths = np.asarray([len(s) for s in trainIDS],dtype=np.int32)
plt.hist(sectionlengths[sectionlengths<1000], 100, density=True, facecolor='g', alpha=0.75)
np.histogram(sectionlengths[sectionlengths<1000],50)

print(f"Lenght of token rags min:{min(sectionlengths)} max:{np.max(sectionlengths)} - median:{np.median(sectionlengths)}")
print(f"rags > 1000 tokes:{np.sum(sectionlengths>1000)}")
sectionlengths[:20]

In [ ]:
percent     = np.asarray([ 25,50,75,90,97,99,100]) # %np.arange(101,dtype=np.int)
percentiles = np.percentile(sectionlengths,percent).astype(np.int)
print("%  percentile")
for i in range(len(percent)):print(f"{percent[i]}  {percentiles[i]}")

In [ ]:
from languagemodelloader import *
#i have an issue with passing pad_idx

#uses nTrainToks, nValidToks to limit the number of tokens used for training/validation
#nTrainToks, nValidToks = 1000,200
#nTrainToks, nValidToks = int(5e5),int(2e5)
nTrainToks, nValidToks = int(5e5),int(1e5)
#nTrainToks, nValidToks = 3000,600
if nTrainToks>0 and nValidToks>0:
    trainIDS_ = trainIDS[0:nTrainToks]
    validIDS_ = validIDS[0:nValidToks]
else:
    trainIDS_ = trainIDS
    validIDS_ = validIDS

#dblm = MyTextLMDataBunch.from_ids( pathTrainValid, vocab, trainIDS_, validIDS_, bptt=50, p_bptt=0, bs=32)
dblm = TextLMDataBunch.from_ids( pathTrainValid, vocab, trainIDS_, validIDS_, bptt=70, p_bptt=1, bs=32)
nTrainToks, nValidToks

In [ ]:
len(dblm.train_ds.vocab.itos)

In [ ]:
dblm.show_batch()

In [ ]:
%time learn = language_model_learner(dblm, drop_mult=0, qrnn=False, pad_token=-1, callback_fns=ShowGraph)

In [ ]:
#%load_ext line_profiler
%time learn.fit_one_cycle(10,2e-3, moms=(0.8,0.7))

In [ ]:
5e5//5//400

In [ ]:
learn.save('model-32k-sentencepiece-vocab')

C:\projects\sentencepiece\src\trainer_interface.cc(363) LOG(INFO) Done: 100% characters are covered.
C:\projects\sentencepiece\src\trainer_interface.cc(373) LOG(INFO) Alphabet size=5727
C:\projects\sentencepiece\src\trainer_interface.cc(374) LOG(INFO) Final character coverage=1
C:\projects\sentencepiece\src\trainer_interface.cc(406) LOG(INFO) Done! preprocessed 998659 sentences.
C:\projects\sentencepiece\src\unigram_model_trainer.cc(129) LOG(INFO) Making suffix array...
[I 14:36:20.223 NotebookApp] Saving file at /nlp/nonsens scratchpad.ipynb
C:\projects\sentencepiece\src\unigram_model_trainer.cc(133) LOG(INFO) Extracting frequent sub strings...
C:\projects\sentencepiece\src\unigram_model_trainer.cc(184) LOG(INFO) Initialized 1000000 seed sentencepieces
C:\projects\sentencepiece\src\trainer_interface.cc(412) LOG(INFO) Tokenizing input sentences with whitespace: 998659
C:\projects\sentencepiece\src\trainer_interface.cc(421) LOG(INFO) Done! 2058688
C:\projects\sentencepiece\src\unigram_model_trainer.cc(472) LOG(INFO) Using 2058688 sentences for EM training